# RESEARCH NOTEBOOK --> MACDBB

In [ ]:
# This is necessary to recognize the modules
import os
import sys
import warnings

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(root_path)

In [ ]:
import pandas_ta as ta  # noqa: F401

from core.data_sources import CLOBDataSource

# Initialize the data source
clob = CLOBDataSource()

In [ ]:
# Define the parameters
exchange = "binance_perpetual"
trading_pair = "ETH-USDT"
timeframe = "3m"
days = 4

In [ ]:
# Get the candles
candles = await clob.get_candles_last_days(exchange, trading_pair, timeframe, days, from_trades=False)

In [ ]:
candles.data.head(2)

In [ ]:
candles.plot(type="returns")

In [ ]:
# Bollinger Bands
bb_length = 100
bb_std = 1.5
bb_long_threshold = 0
bb_short_threshold = 1

# MACD
macd_fast = 21
macd_slow = 42
macd_signal = 9

# Add indicators
candles_df = candles.data
candles_df.ta.bbands(length=bb_length, std=bb_std, append=True)
candles_df.ta.macd(fast=macd_fast, slow=macd_slow, signal=macd_signal, append=True)
candles_df[
    [
        "timestamp",
        "close",
        f"BBU_{bb_length}_{bb_std}",
        f"BBL_{bb_length}_{bb_std}",
        f"MACD_{macd_fast}_{macd_slow}_{macd_signal}",
        f"MACDs_{macd_fast}_{macd_slow}_{macd_signal}",
        f"MACDh_{macd_fast}_{macd_slow}_{macd_signal}",
    ]
].tail(5)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.03, subplot_titles=(trading_pair, "MACD"), row_heights=[0.7, 0.3]
)

# Add candlestick
fig.add_trace(
    go.Candlestick(
        x=candles_df.index,
        open=candles_df["open"],
        high=candles_df["high"],
        low=candles_df["low"],
        close=candles_df["close"],
        name="OHLC",
    ),
    row=1,
    col=1,
)

# Add Bollinger Bands
bb_upper = f"BBU_{bb_length}_{bb_std}"
bb_lower = f"BBL_{bb_length}_{bb_std}"
fig.add_trace(
    go.Scatter(x=candles_df.index, y=candles_df[bb_upper], line=dict(color="rgba(173, 204, 255, 0.7)"), name="BB Upper"),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=candles_df.index,
        y=candles_df[bb_lower],
        line=dict(color="rgba(173, 204, 255, 0.7)"),
        fill="tonexty",
        fillcolor="rgba(173, 204, 255, 0.1)",
        name="BB Lower",
    ),
    row=1,
    col=1,
)

# Add MACD
macd = f"MACD_{macd_fast}_{macd_slow}_{macd_signal}"
macd_s = f"MACDs_{macd_fast}_{macd_slow}_{macd_signal}"
macd_hist = f"MACDh_{macd_fast}_{macd_slow}_{macd_signal}"

fig.add_trace(go.Scatter(x=candles_df.index, y=candles_df[macd], line=dict(color="#00FFFF", width=2), name="MACD"), row=2, col=1)
fig.add_trace(
    go.Scatter(x=candles_df.index, y=candles_df[macd_s], line=dict(color="#FFA500", width=2), name="Signal"), row=2, col=1
)
fig.add_trace(
    go.Bar(
        x=candles_df.index,
        y=candles_df[macd_hist],
        name="Histogram",
        marker_color=candles_df[macd_hist].apply(lambda x: "#00FF00" if x >= 0 else "#FF0000"),
    ),
    row=2,
    col=1,
)

# Update layout for dark theme
fig.update_layout(
    title=f"{exchange} - {trading_pair} - {timeframe}",
    width=1200,
    height=800,
    font=dict(color="#e1e1e1"),
    plot_bgcolor="#1e1e1e",
    paper_bgcolor="#1e1e1e",
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor="rgba(0,0,0,0)"),
    yaxis=dict(title="Price"),
    yaxis2=dict(title="MACD", showgrid=False),
    showlegend=False,
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor="#323232", zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor="#323232", zeroline=False)

# Show the plot
fig.show()

In [ ]:
# Generate signal

candles_df["signal"] = 0
bbp = candles_df[f"BBP_{bb_length}_{bb_std}"]
macdh = candles_df[f"MACDh_{macd_fast}_{macd_slow}_{macd_signal}"]
macd = candles_df[f"MACD_{macd_fast}_{macd_slow}_{macd_signal}"]

long_condition = (bbp < bb_long_threshold) & (macdh > 0) & (macd < 0)
short_condition = (bbp > bb_short_threshold) & (macdh < 0) & (macd > 0)

candles_df.loc[long_condition, "signal"] = 1
candles_df.loc[short_condition, "signal"] = -1

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    subplot_titles=("OHLC with BB", "MACD", "Signal"),
    row_heights=[0.6, 0.2, 0.2],
)

# Add candlestick
fig.add_trace(
    go.Candlestick(
        x=candles_df.index,
        open=candles_df["open"],
        high=candles_df["high"],
        low=candles_df["low"],
        close=candles_df["close"],
        name="Candlesticks",
    ),
    row=1,
    col=1,
)

# Add Bollinger Bands
bb_upper = f"BBU_{bb_length}_{bb_std}"
bb_lower = f"BBL_{bb_length}_{bb_std}"
fig.add_trace(
    go.Scatter(x=candles_df.index, y=candles_df[bb_upper], line=dict(color="rgba(173, 204, 255, 0.7)"), name="BB Upper"),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=candles_df.index,
        y=candles_df[bb_lower],
        line=dict(color="rgba(173, 204, 255, 0.7)"),
        fill="tonexty",
        fillcolor="rgba(173, 204, 255, 0.1)",
        name="BB Lower",
    ),
    row=1,
    col=1,
)

# Add MACD
macd_line = f"MACD_{macd_fast}_{macd_slow}_{macd_signal}"
signal_line = f"MACDs_{macd_fast}_{macd_slow}_{macd_signal}"
hist_line = f"MACDh_{macd_fast}_{macd_slow}_{macd_signal}"

fig.add_trace(
    go.Scatter(x=candles_df.index, y=candles_df[macd_line], line=dict(color="#00FFFF", width=2), name="MACD"), row=2, col=1
)
fig.add_trace(
    go.Scatter(x=candles_df.index, y=candles_df[signal_line], line=dict(color="#FFA500", width=2), name="Signal"), row=2, col=1
)
fig.add_trace(
    go.Bar(
        x=candles_df.index,
        y=candles_df[hist_line],
        name="Histogram",
        marker_color=candles_df[hist_line].apply(lambda x: "#00FF00" if x >= 0 else "#FF0000"),
    ),
    row=2,
    col=1,
)

# Add the signal line
fig.add_trace(
    go.Scatter(x=candles_df.index, y=candles_df["signal"], mode="lines", name="Signal", line=dict(color="white")), row=3, col=1
)

# Update layout for dark theme
fig.update_layout(
    title=f"{exchange} - {trading_pair} - {timeframe}",
    width=1200,
    height=800,
    font=dict(color="#e1e1e1"),
    plot_bgcolor="#1e1e1e",
    paper_bgcolor="#1e1e1e",
    xaxis_rangeslider_visible=False,
    legend=dict(bgcolor="rgba(0,0,0,0)"),
    yaxis=dict(title="Price"),
    yaxis2=dict(title="MACD", showgrid=False),
    yaxis3=dict(title="Signal", showgrid=False),
    showlegend=False,
)

# Update axes
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor="#323232", zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor="#323232", zeroline=False)

# Show the plot
fig.show()

# CONCLUSION

In this notebook, we have implemented a strategy combining the MACD (Moving Average Convergence Divergence) indicator with Bollinger Bands. We've visualized these indicators along with the price data and generated signals based on their interactions. This approach provides a solid foundation for our trading strategy.
 
## Key components of our strategy include:
 1. MACD for trend identification
 2. Bollinger Bands for volatility measurement and potential reversal points
 3. A signal line derived from the combination of these indicators
 
 The next step is to backtest this strategy to evaluate its profitability and robustness. For this purpose, we have created a controller file named `macd_bb.py` in this folder. This file implements the logic we've developed here, allowing us to conduct comprehensive backtests in the subsequent notebook.